In [34]:
import time
from sklearn.datasets import load_digits
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
import catboost as cb
import plotly.express as px
from xgboost import XGBClassifier
from sklearn import metrics


if __name__ == '__main__':
    data = load_digits(as_frame=False)
    predicts = data.data
    target = data.target
    print('\nТаблица предикторов:\n')
    print(predicts)
    print('\nТаблица целевых переменных:\n')
    print(target)
    print('\nРазмерность таблицы предикторов:\n')
    print(predicts.shape)
    print('\nРазмерность таблицы целеых переменных:\n')
    print(target.shape)
    A_train, A_test, y_train, y_test = train_test_split(predicts, target, train_size=0.8)
    print('\nРазделение на тренировочную и тестовую выборки:\n')
    print(A_train.shape)
    print(A_test.shape)
    time_start = time.time()
    print('\nОбучение случайного леса без критерия останова:\n')
    random_forest = RandomForestClassifier(max_depth=15,min_samples_split=10).fit(A_train,y_train)

    y_preds_d = random_forest.predict(A_train)
    y_pred = random_forest.predict(A_test)
    foresttrain=f1_score(y_preds_d,y_train, average='macro')
    foresttest=f1_score(y_pred,y_test, average='macro')
    print("RandomForest\nTime used: ", time.time() - time_start)
    print('F1 мера для тренировочных данных {} % '.format(round((foresttrain * 100), 6)))
    print('F1 мера для тестовых данных {} % '.format(round((foresttest * 100), 6)))

    print('\nОбучение случайного используя критерия : перебор параметров и для каждой'
          'комбинации обучить модели, затем выбрать лучшую. Это можно сделать с помощью'
'GridSearchCV.\n')
    time_start = time.time()
    random_forest = RandomForestClassifier()
    params_grid = {"max_depth":[12,18], "min_samples_leaf":[1,10], "min_samples_split":[6,12]}
    grid_search_random_forest = GridSearchCV(estimator=random_forest, param_grid=params_grid, scoring="f1_macro",cv=4)

    grid_search_random_forest.fit(A_train,y_train)
    best_model = grid_search_random_forest.best_estimator_
    y_preds_d = best_model.predict(A_train)
    y_pred = best_model.predict(A_test)
    trainpercent=f1_score(y_preds_d,y_train, average='macro')
    testpercent=f1_score(y_pred,y_test, average='macro')
    print(best_model)
    print("RandomForest\nTime used: ", time.time() - time_start)
    print('F1 мера для тренировочных данных{} % '.format(round((trainpercent * 100), 6)))
    print('F1 мера для тестовых данных {} %'.format(round((testpercent * 100), 6)))
    fig = px.imshow(confusion_matrix(y_test, y_pred), text_auto=True)
    fig.update_layout(xaxis_title='Target', yaxis_title='Prediction')
    fig.show()
    print(classification_report(y_test, y_pred))

    time_start = time.time()
    model_catboost_clf = cb.CatBoostClassifier(iterations=1000, task_type="GPU", devices='0')
    model_catboost_clf.fit(A_train, y_train)
    y_preds_t = model_catboost_clf.predict(A_train, task_type="CPU")
    y_preds = model_catboost_clf.predict(A_test, task_type="CPU")
    print("XgBoost\nTime used: ", time.time() - time_start)
    f1score=f1_score(y_preds_t,y_train, average='macro')
    print('F1 мера для тренировочных данных {} %  '.format(round((f1score * 100), 6)))
    print('F1 мера для тестовых данных {} %  '.format(round((f1score * 100), 6)))
    fig = px.imshow(confusion_matrix(y_test, y_preds), text_auto=True)
    fig.update_layout(xaxis_title='Target', yaxis_title='Prediction')
    fig.show()
    print(classification_report(y_test, y_preds))

    print('\n\nВыполнение третьего задания:\n')

    start = time.perf_counter()

    xgbCl = XGBClassifier()
    xgbCl.fit(A_train, y_train)
    test_predict = xgbCl.predict(A_test)
    xgbCl_score = xgbCl.score(A_test, y_test)
    time_start = time.time()
    print("Точность XGBoostClassifier модели равна: {} %".format(round((xgbCl_score * 100), 6)))
    print("\nМатрица ошибок XGBoostClassifier модели :")
    fig = px.imshow(confusion_matrix(y_test, test_predict), text_auto=True)
    fig.update_layout(xaxis_title='Target', yaxis_title='Prediction')
    fig.show()
    #print(metrics.confusion_matrix(y_test, test_predict))

    y_predict = xgbCl.predict_proba(A_test)
    # print('Предсказанные данные:\n', y_predict)
    print("XgBoost\nTime used: ", time.time() - time_start)





Таблица предикторов:

[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]

Таблица целевых переменных:

[0 1 2 ... 8 9 8]

Размерность таблицы предикторов:

(1797, 64)

Размерность таблицы целеых переменных:

(1797,)

Разделение на тренировочную и тестовую выборки:

(1437, 64)
(360, 64)

Обучение случайного леса без критерия останова:

RandomForest
Time used:  0.3370356559753418
F1 мера для тренировочных данных 100.0 % 
F1 мера для тестовых данных 96.447517 % 

Обучение случайного используя критерия : перебор параметров и для каждойкомбинации обучить модели, затем выбрать лучшую. Это можно сделать с помощьюGridSearchCV.

RandomForestClassifier(max_depth=18, min_samples_split=6)
RandomForest
Time used:  30.737576246261597
F1 мера для тренировочных данных100.0 % 
F1 мера для тестовых данных 97.085146 %


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       0.98      1.00      0.99        43
           2       0.97      1.00      0.99        38
           3       1.00      0.94      0.97        36
           4       0.97      0.95      0.96        38
           5       0.95      1.00      0.97        39
           6       1.00      0.97      0.98        32
           7       0.97      1.00      0.98        32
           8       0.88      0.91      0.89        32
           9       1.00      0.93      0.97        30

    accuracy                           0.97       360
   macro avg       0.97      0.97      0.97       360
weighted avg       0.97      0.97      0.97       360

Learning rate set to 0.069888
0:	learn: 2.0710959	total: 17.4ms	remaining: 17.3s
1:	learn: 1.8986920	total: 33.2ms	remaining: 16.6s
2:	learn: 1.7567134	total: 48.2ms	remaining: 16s
3:	learn: 1.6383114	total: 62.8ms	remaining: 15.6s
4:	learn:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       0.98      0.98      0.98        43
           2       0.97      1.00      0.99        38
           3       1.00      0.94      0.97        36
           4       1.00      0.92      0.96        38
           5       0.95      1.00      0.97        39
           6       1.00      0.97      0.98        32
           7       0.91      1.00      0.96        32
           8       0.97      0.94      0.95        32
           9       0.94      0.97      0.95        30

    accuracy                           0.97       360
   macro avg       0.97      0.97      0.97       360
weighted avg       0.97      0.97      0.97       360



Выполнение третьего задания:

Точность XGBoostClassifier модели равна: 96.944444 %

Матрица ошибок XGBoostClassifier модели :


XgBoost
Time used:  0.07900118827819824
